# Adler phasecurve models
This notebook demonstrates how Adler implements phasecurve models. An example object with photometric observations is loaded. We can create a phasecurve model object from the SSObject parameters associated with this object. We can also fit a phasecurve model of our choice to the observations.

In [ ]:
from adler.dataclasses.AdlerPlanetoid import AdlerPlanetoid
from adler.science.PhaseCurve import PhaseCurve

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u

In [ ]:
# ssObjectId of object to analyse
ssoid = "8268570668335894776"

In [ ]:
# retrieve the object data via adler

# # here we use an offline SQL database which contains the observations of the sso
fname = "../../tests/data/testing_database.db"
planetoid = AdlerPlanetoid.construct_from_SQL(ssoid, sql_filename=fname)

# alternatively we can retrieve the object data directly from the RSP
# planetoid = AdlerPlanetoid.construct_from_RSP(ssoid)

In [ ]:
# inspect the whole object
planetoid.__dict__

In [ ]:
# inspect just the ssObject table
planetoid.SSObject.__dict__

In [ ]:
# retrieve all observations in the r filter
obs_r = planetoid.observations_in_filter("r")

In [ ]:
# inspect the fields available in the observations table
obs_r.__dict__

In [ ]:
# plot the observations as a phasecurve
x_plot = "phaseAngle"
y_plot = "reduced_mag"

x = getattr(obs_r, x_plot)
y = getattr(obs_r, y_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, y, xerr, fmt="o")

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

In [ ]:
# retrieve the phase curve model parameters provided in the ssObject table

sso_r = planetoid.SSObject_in_filter("r")

r_H = sso_r.H
r_G12 = sso_r.G12

pc = PhaseCurve(H=r_H * u.mag, phase_parameter_1=r_G12, model_name="HG12_Pen16")

In [ ]:
# what sbpy model is being used?
pc.model_function

In [ ]:
# set up an array of phase angles to plot the model
alpha = np.linspace(0, np.amax(obs_r.phaseAngle)) * u.deg
alpha

In [ ]:
# calculate the model reduced magnitude over these phase angles
red_mag = pc.ReducedMag(alpha)
red_mag

In [ ]:
# plot the observations with the model phase curve
x_plot = "phaseAngle"
y_plot = "reduced_mag"

x = getattr(obs_r, x_plot)
y = getattr(obs_r, y_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, y, xerr, fmt="o")

ax1.plot(alpha.value, red_mag.value)

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

In [ ]:
# plot the observations as a lightcurve
x_plot = "midPointMjdTai"
y_plot = "reduced_mag"

x = getattr(obs_r, x_plot)
y = getattr(obs_r, y_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, y, xerr, fmt="o")

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

In [ ]:
# do a different phase curve fit to the data
# adler is able to fit different models, and perform more sophisticated fits

In [ ]:
# create a new PhaseCurve object with a different sbpy model
pc_fit = PhaseCurve(H=pc.H, model_name="HG")

In [ ]:
pc_fit.model_function

In [ ]:
# use adler to fit this new phase curve model to the data
pc_fit.FitModel(
    phase_angle=obs_r.phaseAngle * u.deg,
    reduced_mag=obs_r.reduced_mag * u.mag,
    mag_err=obs_r.magErr * u.mag,
)

In [ ]:
# plot the observations with both
x_plot = "phaseAngle"
y_plot = "reduced_mag"

x = getattr(obs_r, x_plot)
y = getattr(obs_r, y_plot)
xerr = obs_r.magErr

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.errorbar(x, y, xerr, fmt="o")

ax1.plot(alpha.value, pc.ReducedMag(alpha).value, label=pc.model_name)
ax1.plot(alpha.value, pc_fit.ReducedMag(alpha).value, label=pc_fit.model_name)

ax1.invert_yaxis()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.legend()

plt.show()

In [ ]:
# # now we would add our calculated values back into planetoid
# planetoid.AdlerSchema.r_H = pc_fit.abs_mag
# planetoid.AdlerSchema.r_G = pc_fit.phase_param